In [ ]:
import numpy as np
import networkx as nx

In [66]:
# Calcola la distanza più breve tra ogni coppia di nodi (shortest path length)
def adjacency_to_shortest_path_distances(A):
    G = nx.from_numpy_array(A)
    length = dict(nx.all_pairs_shortest_path_length(G))
    n = A.shape[0]
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = length[i][j] if j in length[i] else np.inf
    D[D == np.inf] = D[D != np.inf].max() + 1
    return D

def fused_gromov_wasserstein_cost_fixed_transport(X1, X2, C1, C2, alpha):
    n = X1.shape[0]
    T = np.eye(n) # Trasporto identità

    # Features (Wasserstein)
    feature_cost = 0
    for i in range(n):
        feature_cost += np.linalg.norm(X1[i] - X2[i])**2
    feature_cost /= n

    print(feature_cost)
    
    # Struttura (Gromov)
    structure_cost = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                for l in range(n):
                    structure_cost += (C1[i,k] - C2[j,l])**2 * T[i,j] * T[k,l]
    structure_cost /= n**2

    print(structure_cost)


    return alpha * structure_cost + (1 - alpha) * feature_cost

In [67]:
# Esempio dati

A1 = np.array([[0,1,0],
               [1,0,1],
               [0,1,0]])

A2 = np.array([[0,1,1],
               [1,0,0],
               [1,0,0]])

A2 = np.array([[0,0,1],
               [0,0,0],
               [1,0,0]])

X1 = np.array([[0.5, 1.0],
               [1.0, 0.0],
               [0.5, 0.5]])

X2 = np.array([[0.0, 1.0],
               [1.0, 1.0],
               [0.5, 0.0]])

X2 = np.array([[1.0, 1.0],
               [1.0, 0.0],
               [0.5, 0.5]])

C1 = adjacency_to_shortest_path_distances(A1)
C2 = adjacency_to_shortest_path_distances(A2)

alpha = 0.8
cost = fused_gromov_wasserstein_cost_fixed_transport(X1, X2, C1, C2, alpha)

print("Costo FGW con trasporto identità:", cost)

0.08333333333333333
0.6666666666666666
Costo FGW con trasporto identità: 0.55


In [71]:
import numpy as np
import networkx as nx

# Dati forniti (corretto terzo grafo)
A1 = np.array([[0,1,0],
               [1,0,1],
               [0,1,0]])

A2 = np.array([[0,1,1],
               [1,0,0],
               [1,0,0]])

A3 = np.array([[0,0,1],
               [0,0,0],
               [1,0,0]])

X1 = np.array([[0.5, 1.0],
               [1.0, 0.0],
               [0.5, 0.5]])

X2 = np.array([[0.0, 1.0],
               [1.0, 1.0],
               [0.5, 0.0]])

X3 = np.array([[1.0, 1.0],
               [1.0, 0.0],
               [0.5, 0.5]])

def adjacency_to_shortest_path_distances(A):
    G = nx.from_numpy_array(A)
    length = dict(nx.all_pairs_shortest_path_length(G))
    n = A.shape[0]
    D = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            D[i, j] = length[i][j] if j in length[i] else np.inf
    D[D == np.inf] = D[D != np.inf].max() + 1
    return D

def fused_gromov_wasserstein_cost_fixed_transport(X1, X2, C1, C2, alpha):
    n = X1.shape[0]
    T = np.eye(n)  # trasporto identità

    structure_cost = np.mean((C1 - C2)**2)
    feature_cost = np.mean(np.linalg.norm(X1 - X2, axis=1)**2)

    return alpha * structure_cost + (1 - alpha) * feature_cost

def compute_baricentro_fixed_transport(X_list, C_list, alpha=0.5, max_iter=10, tol=1e-5):
    m = len(X_list)
    n = X_list[0].shape[0]
    d = X_list[0].shape[1]

    X_bar = np.mean(np.array(X_list), axis=0)
    C_bar = np.mean(np.array(C_list), axis=0)

    for it in range(max_iter):
        X_bar_old = X_bar.copy()
        C_bar_old = C_bar.copy()

        # Aggiorniamo come media semplice (T=I)
        X_bar = np.mean(np.array(X_list), axis=0)
        C_bar = np.mean(np.array(C_list), axis=0)

        diff_X = np.linalg.norm(X_bar - X_bar_old) / np.linalg.norm(X_bar_old)
        diff_C = np.linalg.norm(C_bar - C_bar_old) / np.linalg.norm(C_bar_old)

        print(f"Iter {it}: diff_X = {diff_X:.6f}, diff_C = {diff_C:.6f}")

        if diff_X < tol and diff_C < tol:
            print("Converged")
            break

    return X_bar, C_bar

# Calcolo distanze strutturali
C1 = adjacency_to_shortest_path_distances(A1)
C2 = adjacency_to_shortest_path_distances(A2)
C3 = adjacency_to_shortest_path_distances(A3)

X_list = [X1, X2, X3]
C_list = [C1, C2, C3]

X_bar, C_bar = compute_baricentro_fixed_transport(X_list, C_list, alpha=0.8)

print("Feature baricentro:\n", X_bar)
print("Struttura baricentro:\n", C_bar)

# Calcolo costo FGW baricentro vs primo grafo
cost = fused_gromov_wasserstein_cost_fixed_transport(X_bar, X1, C_bar, C1, alpha=0.8)
print(f"Costo FGW baricentro vs G1: {cost:.4f}")

cost = fused_gromov_wasserstein_cost_fixed_transport(X_bar, X2, C_bar, C2, alpha=0.8)
print(f"Costo FGW baricentro vs G2: {cost:.4f}")

cost = fused_gromov_wasserstein_cost_fixed_transport(X_bar, X3, C_bar, C3, alpha=0.8)
print(f"Costo FGW baricentro vs G3: {cost:.4f}")


Iter 0: diff_X = 0.000000, diff_C = 0.000000
Converged
Feature baricentro:
 [[0.5        1.        ]
 [1.         0.33333333]
 [0.5        0.33333333]]
Struttura baricentro:
 [[0.         1.33333333 1.33333333]
 [1.33333333 0.         1.66666667]
 [1.33333333 1.66666667 0.        ]]
Costo FGW baricentro vs G1: 0.1870
Costo FGW baricentro vs G2: 0.1130
Costo FGW baricentro vs G3: 0.1444


In [70]:
print(C1, "\n\n", C2, "\n\n", C3)

[[0. 1. 2.]
 [1. 0. 1.]
 [2. 1. 0.]] 

 [[0. 1. 1.]
 [1. 0. 2.]
 [1. 2. 0.]] 

 [[0. 2. 1.]
 [2. 0. 2.]
 [1. 2. 0.]]
